# Insight Data Science - Scraped Fellow Data

In Anaconda prompt, enter 'pip install lxml' and 'pip install requests'

In [1]:
from lxml import html
import requests
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline
import math
from itertools import groupby
import collections

## Insight Health Data Science Fellows

### This program currently operates on site in either Boston, MA or Seattle, WA (no Remote option)

In [2]:
# Scrape data from webpage
page = requests.get('http://insighthealthdata.com/fellows')
tree = html.fromstring(page.content)

Which companies have hired the most Health fellows?

In [3]:
#Extract a list of names:
health_name = tree.xpath('//div[@class="fellow_name"]/text()')
#Extract a list of companies:
health_company = tree.xpath('//div[@class="fellow_company"]/text()')
#Extract job title:
health_title = tree.xpath('//div[@class="toottip_title"]/text()')
#Extract academic background: 
health_bckgrd = tree.xpath('//div[@class="tooltip_background"]/text()')
#Extract project description:
health_project = tree.xpath('//div[@class="tooltip_project"]/text()')

In [4]:
#Combine fields into data frame
health_fellows = pd.DataFrame(data=[health_company, health_title, health_bckgrd, health_project]).transpose()

In [5]:
#Rename columns
health_fellows.columns=['Company','Title','Background','Project']

In [6]:
#How many fellows in this data set?
num = len(health_fellows)
print("There are %d fellows in this dataset" %num)

There are 91 fellows in this dataset


In [7]:
#Check first five rows of data
health_fellows[:5]

,Company,Title,Background,Project
0,Seven Bridges,Scientific Product Manager,"Neuroscience, Johns Hopkins University, PhD",Just-go-ah: Detecting Parkinson's disease from...
1,Immuneering,Scientist,"Systems Biology, Harvard, Postdoc",Oncology Oracle: Predict Cancer Thereaputic Re...
2,Tamr,Data Scientist,"Cognitive Neuroscience, McGill University, PhD",Steps4health: Leveraging Fitbit data for autom...
3,The Broad Institute,Computational Biologist,"Cell Biology, Harvard, Postdoc",OncoPredictor: Predicting patient response to ...
4,Dana Farber Cancer Institute,Bioinformatician,"Bioinformatics, University of Massachusetts Me...",PAINS Train: Identifying false positive result...


Need to split the Background field into separate components (field, institution, degree)

In [8]:
hf = health_fellows.join(health_fellows['Background'].str.split(',', expand=True).rename(columns={0:'Field', 1:'Institution',2:'Degree'}))

In [9]:
#Check first five rows
hf[:5]

,Company,Title,Background,Project,Field,Institution,Degree,3,4
0,Seven Bridges,Scientific Product Manager,"Neuroscience, Johns Hopkins University, PhD",Just-go-ah: Detecting Parkinson's disease from...,Neuroscience,Johns Hopkins University,PhD,None,None
1,Immuneering,Scientist,"Systems Biology, Harvard, Postdoc",Oncology Oracle: Predict Cancer Thereaputic Re...,Systems Biology,Harvard,Postdoc,None,None
2,Tamr,Data Scientist,"Cognitive Neuroscience, McGill University, PhD",Steps4health: Leveraging Fitbit data for autom...,Cognitive Neuroscience,McGill University,PhD,None,None
3,The Broad Institute,Computational Biologist,"Cell Biology, Harvard, Postdoc",OncoPredictor: Predicting patient response to ...,Cell Biology,Harvard,Postdoc,None,None
4,Dana Farber Cancer Institute,Bioinformatician,"Bioinformatics, University of Massachusetts Me...",PAINS Train: Identifying false positive result...,Bioinformatics,University of Massachusetts Medical School,PhD,None,None


In [10]:
#Check summary info for all variables
hf.describe()

,Company,Title,Background,Project,Field,Institution,Degree,3,4
count,89,90,91,91,91,87,84,6,1
unique,63,38,89,91,58,64,17,6,1
top,Seven Bridges,Data Scientist,"Physics, Harvard University, PhD",Pitcher Prognosis: Tracking performance to cat...,Physics,Harvard University,PhD,University of Texas at Austin,PhD
freq,9,29,2,1,10,6,33,1,1


Need to address the last two columns. There were multiple commas in the Background variable that caused this shift.

In [11]:
#Examine column "3" values
collections.Counter(hf[3])

Counter({None: 85,
         '\xa0PhD': 1,
         ' PhD': 1,
         ' University of Texas at Austin': 1,
         ' Principal Investigator ': 1,
         ' Postdoc ': 1,
         ' PhD ': 1})

In [12]:
# Find rows where data shifted into column 3
hf.loc[hf[3].isin(['\xa0PhD',' PhD',' University of Texas at Austin',' Principal Investigator ',' Postdoc ',' PhD '])]

,Company,Title,Background,Project,Field,Institution,Degree,3,4
26,Wayfair,Data Scientist,"Mathematics, University of Minnesota, Twin Cit...",Limelight: Optimizing bioprocessing experiments,Mathematics,University of Minnesota,Twin Cities,PhD,None
36,Wayfair,Data Scientist,"Statistical Genetics, University of Cambridge,...",Genomix: Mixing genomic data for an Alzheimer'...,Statistical Genetics,University of Cambridge,Sanger Institute,PhD,None
53,DATAIKU Inc.,Data Scientist,"Ecology, Evolution, and Behavior, University o...",The Lyme Forecast: Predicting the spread of Ly...,Ecology,Evolution,and Behavior,University of Texas at Austin,PhD
59,Accenture,Senior Mathematician,"Theoretical Physics & Population Biology, Univ...",HealthConnect.Online: Automating online inform...,Theoretical Physics & Population Biology,University California,Davis,Principal Investigator,None
69,Counsyl,Data Scientist,"Biochemistry, University of California, Berkel...",PleaseInsure: Helping health insurance profess...,Biochemistry,University of California,Berkeley,Postdoc,None
90,None,None,"Population Genomics, University of Massachuset...",Trust your gut: Predict colorectal cancer risk...,Population Genomics,University of Massachusetts,Amherst,PhD,None


In [13]:
# Manually recode six rows of data
hf.at[26, 'Institution'] = 'University of Minnesota, Twin Cities'
hf.at[26, 'Degree'] = 'PhD'
hf.at[36, 'Institution'] = 'University of Cambridge, Sanger Institute'
hf.at[36, 'Degree'] = 'PhD'
hf.at[53, 'Field'] = 'Ecology, Evoluation, and Behavior'
hf.at[53, 'Institution'] = 'University of Texas at Austin'
hf.at[53, 'Degree'] = 'PhD'
hf.at[59, 'Institution'] = 'University of California, Davis'
hf.at[59, 'Degree'] = 'Postdoc'
hf.at[69, 'Institution'] = 'University of California, Berkeley'
hf.at[69, 'Degree'] = 'Postdoc'
hf.at[90, 'Institution'] = 'University of Massachusetts, Amherst'
hf.at[90, 'Degree'] = 'PhD'

In [14]:
hf.Degree = hf['Degree'].str.strip()

In [15]:
collections.Counter(hf.Degree)

Counter({'PhD': 41,
         'Postdoc': 36,
         'Research Fellow': 1,
         'Research Engineer': 1,
         'Research Scientist': 1,
         None: 7,
         'Research Associate': 1,
         'Clinical Assistant Professor': 1,
         'Berkeley': 1,
         'Livermore': 1})

In [16]:
# Find rows where Degree = None
hf.loc[hf.Degree.isnull()]

,Company,Title,Background,Project,Field,Institution,Degree,3,4
29,Biogen,Data Scientist,Theoretical Astrophysics Swinburne University ...,MediTweeter: Discovering medical professionals...,Theoretical Astrophysics Swinburne University ...,PhD,None,None,None
64,Driver Group,Data Scientist,PhD in Genomics and Molecular Biology from UC ...,"SkinClarity, a web app to provide acne medicat...",PhD in Genomics and Molecular Biology from UC ...,None,None,None,None
65,Insight Data Science,Computational Biologist,PhD in Biophysics from University of Chicago,"""Sexpertise"" Content ranking strategy to provi...",PhD in Biophysics from University of Chicago,None,None,None,None
66,Seven Bridges,"Program Director, Data Science",PhD in Physics with a concentration in bioinfo...,"""eigendoctor.com,"" an app to help patients cho...",PhD in Physics with a concentration in bioinfo...,Santa Barbara.,None,None,None
67,Twist,Scientific Project Manager,PhD in Molecular and Cell Biology from UC Berk...,Developed novel strategy to cluster flow cytom...,PhD in Molecular and Cell Biology from UC Berk...,None,None,None,None
68,Grand Rounds,Data Scientist,PhD in Epidemiology and Public Health at UCLA ...,"""www.no-squitoes.com,"" A general malaria anoma...",PhD in Epidemiology and Public Health at UCLA ...,None,None,None,None
70,Doximity,Data Scientist,"Experimental Particle Physics, University of C...",IsThisMetastasis: Machine learning methods for...,Experimental Particle Physics,University of California Santa Cruz. PhD,None,None,None


In [17]:
# Get full value of problem cells
#hf.at[29, 'Field']
#hf.at[64, 'Field']
#hf.at[66, 'Field']
hf.at[68, 'Field']

'PhD in Epidemiology and Public Health at UCLA and Masters of Public Health at Columbia.'

In [18]:
# Manually recode seven additional rows of data (index 29, 64, 65, 66, 67, 68, 70)
hf.at[29, 'Field'] = 'Theoretical Astrophysics'
hf.at[29, 'Institution'] = 'Swinburne University of Technology'
hf.at[29, 'Degree'] = 'PhD'

hf.at[64, 'Field'] = 'Genomics and Molecular Biology'
hf.at[64, 'Institution'] = 'UC Berkeley'
hf.at[64, 'Degree'] = 'PhD'

hf.at[65, 'Field'] = 'Biophysics'
hf.at[65, 'Institution'] = 'University of Chicago'
hf.at[65, 'Degree'] = 'PhD'

hf.at[66, 'Field'] = 'Physics'
hf.at[66, 'Institution'] = 'University of California, Santa Barbara'
hf.at[66, 'Degree'] = 'PhD'

hf.at[67, 'Field'] = 'Molecular and Cell Biology'
hf.at[67, 'Institution'] = 'UC Berkeley'
hf.at[67, 'Degree'] = 'PhD'

hf.at[68, 'Field'] = 'Epidemiology and Public Health'
hf.at[68, 'Institution'] = 'UCLA'
hf.at[68, 'Degree'] = 'PhD'

hf.at[70, 'Institution'] = 'University of California Santa Cruz'
hf.at[70, 'Degree'] = 'PhD'

In [19]:
collections.Counter(hf.Degree)

Counter({'Berkeley': 1,
         'Clinical Assistant Professor': 1,
         'Livermore': 1,
         'PhD': 48,
         'Postdoc': 36,
         'Research Associate': 1,
         'Research Engineer': 1,
         'Research Fellow': 1,
         'Research Scientist': 1})

In [20]:
# Find rows where Degree is not PhD or beyond
hf.loc[hf['Degree'].isin(['Berkeley','Livermore'])]

,Company,Title,Background,Project,Field,Institution,Degree,3,4
62,Ancestry,Computational Biologist & Data Scientist,"Post-doctoral Researcher, Computational Neuros...",iSonogram: Image analysis solution for monitor...,Post-doctoral Researcher,Computational Neuroscience at University of C...,Berkeley,None,None
63,Color Genomics,Computational Biologist,"Postdoctoral Scientist, Genomics & Computation...",iPredictome: Predicting preterm childbirth fro...,Postdoctoral Scientist,Genomics & Computational Biology at Sandia Na...,Livermore,None,None


In [21]:
# See full value of Background at these rows
#hf.at[62, 'Background']
hf.at[63, 'Background']

'Postdoctoral Scientist, Genomics & Computational Biology at Sandia National Laboratories, Livermore '

In [22]:
#Manually recode rows
hf.at[62, 'Field'] = 'Computational Neuroscience'
hf.at[62, 'Institution'] = 'UC Berkeley'
hf.at[62, 'Degree'] = 'Postdoc'

hf.at[63, 'Field'] = 'Genomics and Computational Biology'
hf.at[63, 'Institution'] = 'Sandia National Laboratories, Livermore'
hf.at[63, 'Degree'] = 'Postdoc'

In [23]:
collections.Counter(hf.Degree)

Counter({'Clinical Assistant Professor': 1,
         'PhD': 48,
         'Postdoc': 38,
         'Research Associate': 1,
         'Research Engineer': 1,
         'Research Fellow': 1,
         'Research Scientist': 1})

*I'm surprised at the amount of data cleaning that was required to fix these fields; this was really good practice for data wrangling and pandas functions*

Now let's work with the Institution column

In [24]:
#collections.Counter(hf.Institution)
# Strip white space
hf.Institution = hf['Institution'].str.strip()
collections.Counter(hf.Institution)

Counter({'BAE Systems': 1,
         "Boston Children's Hostpial": 1,
         'Boston College': 1,
         'Boston University': 2,
         "Brigham & Women's Hospital & Harvard Medical School": 1,
         'Brown University': 1,
         'CUNY': 1,
         'Centre National de la Researche Scientifique': 1,
         'Colorado State University': 1,
         'Columbia University': 1,
         'Cornell University': 6,
         'Drexel University': 1,
         'Duke University': 1,
         'Emory University': 1,
         'Harvard': 6,
         'Harvard Medical School': 1,
         'Harvard University': 6,
         'Johns Hopkins School of Public Health': 1,
         'Johns Hopkins University': 2,
         'MIT': 5,
         'Massachusetts Institute of Technology': 1,
         'McGill University': 1,
         'New York University': 1,
         'Northwestern': 1,
         'Northwestern University': 1,
         'Purdue University': 1,
         'Queens University': 1,
         'Rice Univers

In [25]:
#Recode some institution names
hf['Institution'] = hf['Institution'].replace(['Harvard'], 'Harvard University')
hf['Institution'] = hf['Institution'].replace(['UC Berkley', 'University of California, Berkeley'], 'UC Berkeley')
hf['Institution'] = hf['Institution'].replace(['Northwestern'], 'Northwestern University')
hf['Institution'] = hf['Institution'].replace(['The University of Texas at Austin'], 'University of Texas at Austin')
collections.Counter(hf.Institution)

Counter({'BAE Systems': 1,
         "Boston Children's Hostpial": 1,
         'Boston College': 1,
         'Boston University': 2,
         "Brigham & Women's Hospital & Harvard Medical School": 1,
         'Brown University': 1,
         'CUNY': 1,
         'Centre National de la Researche Scientifique': 1,
         'Colorado State University': 1,
         'Columbia University': 1,
         'Cornell University': 6,
         'Drexel University': 1,
         'Duke University': 1,
         'Emory University': 1,
         'Harvard Medical School': 1,
         'Harvard University': 12,
         'Johns Hopkins School of Public Health': 1,
         'Johns Hopkins University': 2,
         'MIT': 5,
         'Massachusetts Institute of Technology': 1,
         'McGill University': 1,
         'New York University': 1,
         'Northwestern University': 2,
         'Purdue University': 1,
         'Queens University': 1,
         'Rice University': 1,
         'Sandia National Laboratories, L

Now, let's look at the Field variable

In [26]:
collections.Counter(hf.Field)

Counter({'Applied Physics': 1,
         'Astrophysics': 2,
         'Biochemistry': 2,
         'Bioengineering': 2,
         'Bioinformatics': 2,
         'Bioinformatics/Genomics': 2,
         'Biological and Biomedical Sciences': 1,
         'Biology': 3,
         'Biomedical Engineering': 2,
         'Biophysics': 4,
         'Brain & Cognitive Sciences': 1,
         'Cell Biology': 1,
         'Cell Biology & Genetics': 1,
         'Chemistry': 1,
         'Cognitive Neuroscience': 3,
         'Computational Biology': 3,
         'Computational Material Sciences': 1,
         'Computational Neuroscience': 1,
         'Computational Polymer Physics': 1,
         'Computer Science': 1,
         'Ecology': 1,
         'Ecology, Evoluation, and Behavior': 1,
         'Electrical and Computer Engineering': 1,
         'Environmental Engineering': 1,
         'Environmental Epidemiology': 1,
         'Environmental Sciences': 1,
         'Epidemiology': 1,
         'Epidemiology and Pub

In [27]:
hf.Title = hf['Title'].str.strip()
collections.Counter(hf.Title)

Counter({'Scientific Product Manager': 1,
         'Scientist': 2,
         'Data Scientist': 40,
         'Computational Biologist': 3,
         'Bioinformatician': 2,
         'Senior Data Engineer Associate': 1,
         'Scientific Project Manager': 3,
         'Senior Statistician': 1,
         'Quantitative User Experience Researcher': 1,
         'R&D Scientist': 3,
         'Research Scientist': 1,
         'Data Science Engineer': 1,
         'Marketing Data Scientist': 1,
         'Program Director, Data Scientist': 2,
         'Health Entrepreneur': 1,
         'Senior Data Scientist': 1,
         'Analyst II - Neuroinformatics': 1,
         'Field Engineer': 3,
         'Senior Analytics Associate': 1,
         'Consultant': 2,
         'Software Engineer': 1,
         'Business Intelligence Consultant': 1,
         'Program Director, Data Science': 2,
         'Associate Director, Data Scientist': 1,
         'Clinical Informatics Analyst': 1,
         'Bioinformatics Scie

In [28]:
# Find where Title = None
hf.loc[hf.Title.isnull()]

,Company,Title,Background,Project,Field,Institution,Degree,3,4
90,None,None,"Population Genomics, University of Massachuset...",Trust your gut: Predict colorectal cancer risk...,Population Genomics,"University of Massachusetts, Amherst",PhD,PhD,None


In [29]:
hf.Company = hf['Company'].str.strip()
collections.Counter(hf.Company)

Counter({'Seven Bridges': 9,
         'Immuneering': 1,
         'Tamr': 4,
         'The Broad Institute': 2,
         'Dana Farber Cancer Institute': 1,
         'Aspen Technology': 1,
         'Athena Health': 1,
         'Understory': 1,
         'Takeda': 1,
         'Foundation Medicine': 1,
         'Rovi': 1,
         'Google': 2,
         'Thomson Reuters': 1,
         'Recorded Future': 1,
         'Wayfair': 9,
         'McKinsey Solutions': 1,
         'Sqrll': 1,
         'Insight Data Science': 3,
         'Higi': 1,
         'TempAltert': 1,
         'Biogen': 1,
         'Humana': 1,
         'Constant Therapy': 1,
         'Cinch Financial': 2,
         'Allen Institute for Brain Science': 1,
         'AbilTo': 1,
         'athenahealth': 1,
         'Schierson': 1,
         'QuantCo': 1,
         'Amazon': 4,
         'Novo Nordiask': 1,
         'Miami Baptist Hospital': 1,
         'Vectra': 1,
         'Merck': 1,
         'Blue Cross and Blue Shield of North Carol

**I found it surprising that Wayfair (eCommerce company) was such a common hirer of Health Data Science Fellows. Upon further research, several members of their Executive Leadership team have previous experience with health data or health companies, and they have an office in Boston so it makes sense.** 

In [30]:
# Find where Company = None
hf.loc[hf.Company.isnull()]

,Company,Title,Background,Project,Field,Institution,Degree,3,4
89,None,Principal Scientist I,"Environmental Epidemiology, University of Mich...",Dentalography: Helping dentists find the optim...,Environmental Epidemiology,University of Michigan,PhD,None,None
90,None,None,"Population Genomics, University of Massachuset...",Trust your gut: Predict colorectal cancer risk...,Population Genomics,"University of Massachusetts, Amherst",PhD,PhD,None


In [31]:
# Delete unneeded columns
hf.drop(hf.columns[[7,8]], axis=1, inplace=True)

In [32]:
hf.describe()

,Company,Title,Background,Project,Field,Institution,Degree
count,89,90,91,91,91,91,91
unique,60,36,89,91,57,59,7
top,Seven Bridges,Data Scientist,"Physics, Harvard University, PhD",Pitcher Prognosis: Tracking performance to cat...,Physics,Harvard University,PhD
freq,9,40,2,1,11,12,48


### Let's do some cross-variable analysis

In [33]:
#Which companies hired Fellows from particular fields?
hf.groupby(['Field', 'Company']).size()

Field                                     Company                                     
Applied Physics                           PatientsLikeMe                                  1
Astrophysics                              Amazon                                          1
                                          Understory                                      1
Biochemistry                              Counsyl                                         1
                                          Google                                          1
Bioengineering                            Allen Institute for Brain Science               1
                                          Seven Bridges                                   1
Bioinformatics                            Dana Farber Cancer Institute                    1
                                          Schireson                                       1
Bioinformatics/Genomics                   Rovi                                       

##  Insight Data Science Fellows 

### Scrape Fellow data from webpage

In [34]:
page_2 = requests.get('http://insightdatascience.com/fellows')
tree_2 = html.fromstring(page_2.content)
#Extract Fellow name:
name=tree_2.xpath('//div[@class="fellow_name"]/text()')
#Extract a list of companies:
company = tree_2.xpath('//div[@class="fellow_company"]/text()')
#Extract job title:
title = tree_2.xpath('//div[@class="toottip_title"]/text()')
#Extract academic background: 
background = tree_2.xpath('//div[@class="tooltip_background"]/text()')
#Extract project title/description
project = tree_2.xpath('//div[@class="tooltip_project"]/text()')

In [13]:
page_2 = requests.get('http://insightdatascience.com/fellows')
tree_2 = html.fromstring(page_2.content)
name = tree_2.xpath('//div[@class="fellow_name"]/text()')
company = tree_2.xpath('//div[@class="fellow_company"]/text()')
title = tree_2.xpath('//div[@class="toottip_title"]/text()')
background = tree_2.xpath('//div[@class="tooltip_background"]/text()')
project = tree_2.xpath('//div[@class="tooltip_project"]/text()')

In [35]:
len_name = len(name)
print("There are %a Insight Data Science Fellows featured on the fellows page." %len_name )

There are 702 Insight Data Science Fellows featured on the fellows page.


### Construct a data frame from all fields

In [37]:
#Construct a data frame from all fields
fellows = pd.DataFrame(data=[company, title, background, project]).transpose()
#Examine first five rows
fellows[:5]

,0,1,2,3
0,Facebook,Data Scientist,"Operations Research, MIT, PhD",Tweet Timeline: Visualizing the Impact of Soci...
1,Adobe,Data Scientist,"Statistics, Stanford, Postdoc",Sentiment Analysis Based on Social Media
2,LinkedIn,Head of Anti-Abuse Engineering,"Mathematics, Stanford, Postdoc",Flight Delay Predictor
3,Capital One,Director of Risk,"Bio-X Games, Stanford, Postdoc",Measuring the Impact of Open Source Coders on ...
4,Proofpoint,Data Scientist,"Physics, UC Riverside, PhD",Analysis of New York Stop-and-Frisk Data


In [38]:
fellows.columns=['Company','Title','Background','Project']

In [40]:
f = fellows.join(fellows['Background'].str.split(',', expand=True).rename(columns={0:'Field', 1:'Institution',2:'Degree'}))
f[:5]

,Company,Title,Background,Project,Field,Institution,Degree,3,4
0,Facebook,Data Scientist,"Operations Research, MIT, PhD",Tweet Timeline: Visualizing the Impact of Soci...,Operations Research,MIT,PhD,None,None
1,Adobe,Data Scientist,"Statistics, Stanford, Postdoc",Sentiment Analysis Based on Social Media,Statistics,Stanford,Postdoc,None,None
2,LinkedIn,Head of Anti-Abuse Engineering,"Mathematics, Stanford, Postdoc",Flight Delay Predictor,Mathematics,Stanford,Postdoc,None,None
3,Capital One,Director of Risk,"Bio-X Games, Stanford, Postdoc",Measuring the Impact of Open Source Coders on ...,Bio-X Games,Stanford,Postdoc,None,None
4,Proofpoint,Data Scientist,"Physics, UC Riverside, PhD",Analysis of New York Stop-and-Frisk Data,Physics,UC Riverside,PhD,None,None


In [41]:
# Like with Health Fellows data set, the split of the Background variable caused data to shift into two additional columns
# Examine column "3" first
collections.Counter(f[3])

Counter({None: 662,
         ' Stanford': 1,
         ' Postdoc': 10,
         ' PhD': 20,
         ' Postdoc\xa0': 1,
         ' Faculty\xa0': 2,
         ' Spain': 1,
         ' Manager': 1,
         ' PhD\xa0': 1,
         ' Clinical Health': 1,
         ' UT Austin': 1,
         ' Texas': 1,
         ' PhD Candidate': 1})

In [43]:
# Find rows where data shifted into column 3
f.loc[f[3].notnull()]

,Company,Title,Background,Project,Field,Institution,Degree,3,4
43,Facebook,Operations Analyst,"Biomedical Genetics, Biochemistry, and Organog...",GeoJobr: Finding a job in today's economy,Biomedical Genetics,Biochemistry,and Organogenesis,Stanford,PhD
62,Juvo,Senior Data Scientist,"Theoretical Astrophysics, Institute for Advanc...",Tweather: Current local weather tweets,Theoretical Astrophysics,Institute for Advanced Study,Princeton,Postdoc,None
72,Intuit,Data Scientist,"Biological Engineering, Computation & Neural S...",InvestWiser: Predicting series A crunch outcome.,Biological Engineering,Computation & Neural Systems,MIT,Postdoc,None
77,Facebook,Data Scientist,"Computational Biology and Biophysics, Kavli In...",RelaxedParking: Parking assistant to avoid tra...,Computational Biology and Biophysics,Kavli Institute for Theoretical Physics,UC Santa Barbara,Postdoc,None
88,Yelp,Engineering Manager,"Experimental Particle Physics, University of C...",OldFaveNewPlace: Find restaurants in a new cit...,Experimental Particle Physics,University of California,Los Angeles,Postdoc,None
89,Stitch Fix,Manager of Data Science,"Computational Atomic, Molecular and Optical Ph...",LinkedInsight: Find the skills and connections...,Computational Atomic,Molecular and Optical Physics,The University of British Columbia,PhD,None
97,Facebook,Ads Analytics,"Astrophysics, Caltech/University of California...",LinkBooksMovies: Book recommendations if you l...,Astrophysics,Caltech/University of California,Santa Barbara,Postdoc,None
104,Apple,Data Scientist,"Plasma Physics, University of Texas, Austin, P...",AdNet: Graph recommender for news domains,Plasma Physics,University of Texas,Austin,Postdoc,None
131,Greenhouse,Data Scientist,"Astrophysics, Niels Bohr Institute, Copenhagen...","nimi: Everyone has a name, your kid needs a gr...",Astrophysics,Niels Bohr Institute,Copenhagen University,Faculty,None
164,CreativeLive,Data Scientist,"Behavioral Economics, ECARES, ULB, Postdoc",How To Strike: Optimizing a directed e-mail ca...,Behavioral Economics,ECARES,ULB,Postdoc,None


In [49]:
# Manually recode data
f.at[43, 'Field'] = 'Biomedical Genetics, Biochemistry, and Organogenesis'
f.at[43, 'Institution'] = 'Stanford'
f.at[43, 'Degree'] = 'PhD'

f.at[62, 'Institution'] = 'Princeton'
f.at[62, 'Degree'] = 'Postdoc'

f.at[72, 'Institution'] = 'MIT'
f.at[72, 'Degree'] = 'Postdoc'

f.at[77, 'Institution'] = 'UC Santa Barbara'
f.at[77, 'Degree'] = 'Postdoc'

f.at[88, 'Institution'] = 'UCLA'
f.at[88, 'Degree'] = 'Postdoc'

f.at[89, 'Institution'] = 'University of British Columbia'
f.at[89, 'Degree'] = 'PhD'

f.at[97, 'Institution'] = 'CalTech/UCSB'
f.at[97, 'Degree'] = 'Postdoc'

f.at[104, 'Institution'] = 'University of Texas, Austin'
f.at[104, 'Degree'] = 'Postdoc'

f.at[131, 'Institution'] = 'Copenhagen University'
f.at[131, 'Degree'] = 'Faculty'

f.at[164, 'Institution'] = 'ULB'
f.at[164, 'Degree'] = 'Postdoc'

f.at[183, 'Institution'] = 'University of Colorado, Boulder'
f.at[183, 'Degree'] = 'Postdoc'

f.at[200, 'Institution'] = 'University of Colorado, Boulder'
f.at[200, 'Degree'] = 'PhD'

f.at[209, 'Institution'] = 'University of Texas, Austin'
f.at[209, 'Degree'] = 'PhD'

f.at[237, 'Institution'] = 'Institute for Space Science, Barcelona, Spain'
f.at[237, 'Degree'] = 'Postdoc'

f.at[293, 'Field'] = 'Analytics & Strategy, Agricultural Microfinance'
f.at[293, 'Institution'] = 'One Acre Fund'
f.at[293, 'Degree'] = 'Manager'

f.at[296, 'Institution'] = 'University of Technology, Sydney, Australia'
f.at[296, 'Degree'] = 'PhD'

f.at[303, 'Institution'] = 'University of Maryland, College Park'
f.at[303, 'Degree'] = 'PhD'

f.at[306, 'Institution'] = 'University of British Columbia'
f.at[306, 'Degree'] = 'PhD'

f.at[308, 'Institution'] = 'University of California, Santa Cruz'
f.at[308, 'Degree'] = 'PhD'

f.at[310, 'Institution'] = 'UCSF'
f.at[310, 'Degree'] = 'Postdoc'

f.at[338, 'Institution'] = 'University of Sydney, Australia'
f.at[338, 'Degree'] = 'PhD'

f.at[382, 'Institution'] = 'University of Colorado, Boulder'
f.at[382, 'Degree'] = 'Faculty'

f.at[393, 'Institution'] = 'University of Illinois, Urbana-Champaign'
f.at[393, 'Degree'] = 'PhD'

f.at[396, 'Institution'] = 'Istituto Nazionale di Fisica Nucleare, Sezione di Roma'
f.at[396, 'Degree'] = 'PhD'

f.at[398, 'Institution'] = 'Georgia Tech, Emory University'
f.at[398, 'Degree'] = 'PhD'

f.at[408, 'Field'] = 'Computation Science, Engineering, & Mathematics'
f.at[408, 'Institution'] = 'University of Texas, Austin'
f.at[408, 'Degree'] = 'PhD'

f.at[452, 'Institution'] = 'UC Riverside'
f.at[452, 'Degree'] = 'PhD'

f.at[468, 'Institution'] = 'UC Riverside'
f.at[468, 'Degree'] = 'Postdoc'

f.at[537, 'Institution'] = 'UCLA'
f.at[537, 'Degree'] = 'PhD'

f.at[562, 'Field'] = 'Computation Science, Engineering, & Mathematics'
f.at[562, 'Institution'] = 'University of Texas, Austin'
f.at[562, 'Degree'] = 'PhD'

f.at[583, 'Institution'] = 'UC Davis'
f.at[583, 'Degree'] = 'PhD'

f.at[584, 'Institution'] = 'Academica Sinica, Taiwan'
f.at[584, 'Degree'] = 'Postdoc'

f.at[586, 'Institution'] = 'UCSD'
f.at[586, 'Degree'] = 'PhD'

f.at[589, 'Institution'] = 'UCSD'
f.at[589, 'Degree'] = 'PhD'

f.at[590, 'Degree'] = 'PhD'

f.at[599, 'Institution'] = 'University of Maryland, College Park'
f.at[599, 'Degree'] = 'PhD Candidate'

f.at[625, 'Institution'] = 'UC Berkeley'
f.at[625, 'Degree'] = 'PhD'

f.at[626, 'Institution'] = 'University of California, Santa Barbara'
f.at[626, 'Degree'] = 'PhD'

f.at[633, 'Institution'] = 'UC Berkeley'
f.at[633, 'Degree'] = 'PhD'

f.at[635, 'Institution'] = 'University of Hawaii, Manoa'
f.at[635, 'Degree'] = 'PhD'

In [50]:
# Now strip spaces from and examine Degree column
f.Degree = f['Degree'].str.strip()
collections.Counter(f.Degree)

Counter({'PhD': 446,
         'Postdoc': 201,
         None: 19,
         'Caltech/California Institute of Technology': 1,
         'Faculty': 6,
         'Systems Engineer': 1,
         'Adjunct Faculty': 1,
         'Staff': 2,
         'Lecturer': 2,
         'Fellow': 1,
         'Associate Research Scientist': 3,
         'Manager': 1,
         'Research Professor': 1,
         'Research Associate': 2,
         'Assistant Professor': 3,
         'Research Scientist': 1,
         'Research Faculty': 2,
         'Principal': 1,
         'Research Project Manager': 1,
         'Research Assistant': 1,
         'Data Scientist': 1,
         'Research Consultant': 1,
         'Assistant Research Scientist': 1,
         'DPhil': 1,
         'PhD Candidate': 1,
         'Principal Scientist': 1,
         'ABD': 1})

In [51]:
# Find rows where Degree has anomalous value
f.loc[f['Degree'].isin(['Caltech/California Institute of Technology'])]

,Company,Title,Background,Project,Field,Institution,Degree,3,4
100,Apple,Data Mining Scientist,"Civil Engineering, Geophysics, Caltech/Califor...",Lenderstanding: Helping lenders help,Civil Engineering,Geophysics,Caltech/California Institute of Technology,None,None


In [52]:
# Recode one row
f.at[100, 'Field'] = 'Civil Engineering, Geophysics'
f.at[100, 'Institution'] = 'Caltech'
# No degree or status listed for this Fellow

#Find rows with null Degree
f.loc[f['Degree'].isnull()]

,Company,Title,Background,Project,Field,Institution,Degree,3,4
42,Facebook,Data Scientist,"Probability Theory and Stochastic Processes,",GameChooser: Intelligent viewing recommendatio...,Probability Theory and Stochastic Processes,,None,None,None
69,LinkedIn,"Manager, Security Data Science",Computational Structural Biology <br> Stanford...,PlacePickr: Helping couples decide where to live,Computational Structural Biology <br> Stanford...,None,None,None,None
199,Facebook,Data Scientist,"Physics, Lawrence Berkeley National Lab Scientist",the Blogtimize/r/: Find your audience and test...,Physics,Lawrence Berkeley National Lab Scientist,None,None,None
243,Wallet.AI,Data Scientist,"Harvard-Smithsonian Center for Astrophysics, P...",Reputon: Rating predictions for Airbnb,Harvard-Smithsonian Center for Astrophysics,Postdoc,None,None,None
361,GNS Healthcare,Research Scientist,"Evident Point Software, Software Developer",Textbooks Unchained: Find free textbooks with ...,Evident Point Software,Software Developer,None,None,None
369,Gartner,Data Scientist,"Observational Astronomy, PhD",cityspinneret: Matching neighborhoods across c...,Observational Astronomy,PhD,None,None,None
501,Nielsen,Data Scientist,"FMC Technologies, Research Scientist - Data An...",CovirTu: A better way to insure farmers agains...,FMC Technologies,Research Scientist - Data Analytics,None,None,None
506,Exelon,Data Scientist,"CERN, Assistant Scientist - High-Energy Physics",Philanthropy Filter: Classifying news articles...,CERN,Assistant Scientist - High-Energy Physics,None,None,None
517,Guggenhiem Partners,Research Fellow,"Trexquant Investment, Trading Strategist",commentizr: Optimize borrower comments to make...,Trexquant Investment,Trading Strategist,None,None,None
528,Uber,Data Scientist,"Astrophysics.University of Minnesota, Postdoc",AboutFace: Identifying and tracking faces in Y...,Astrophysics.University of Minnesota,Postdoc,None,None,None


In [72]:
#f.at[42, 'Background']
#f.at[69, 'Background']
#f.at[199, 'Background']
#f.at[243, 'Background']
#f.at[361, 'Background']
#f.at[369, 'Background']
#f.at[501, 'Background']
#f.at[506, 'Background']
#f.at[517, 'Background']
#f.at[528, 'Background']
#f.at[543, 'Background']
#f.at[547, 'Background']
#f.at[549, 'Background']
#f.at[556, 'Background']
#f.at[559, 'Background']
#f.at[637, 'Background']
#f.at[670, 'Background']
#f.at[670, 'Project']
#f.at[695, 'Background']
f.at[699, 'Background']

'Department of Energy, Science and Technology Policy Fellow'

In [73]:
f.at[42, 'Field'] = 'Probability Theory and Stochastic Processes'

f.at[69, 'Field'] = 'Computational Structural Biology'
f.at[69, 'Institution'] = 'Stanford University'
f.at[69, 'Degree'] = 'Postdoc'

f.at[199, 'Institution'] = 'Lawrence Berkeley National Lab'
f.at[199, 'Degree'] = 'Scientist'

f.at[243, 'Field'] = 'Astrophysics' #a guess
f.at[243, 'Institution'] = 'Harvard-Smithsonian Center for Astrophysics'
f.at[243, 'Degree'] = 'Postdoc'

f.at[361, 'Field'] = ''
f.at[361, 'Institution'] = 'Evident Point Software'
f.at[361, 'Degree'] = 'Software Developer'

f.at[369, 'Institution'] = ''
f.at[369, 'Degree'] = 'PhD'

f.at[501, 'Field'] = ''
f.at[501, 'Institution'] = 'FMC Technologies'
f.at[501, 'Degree'] = 'Research Scientist - Data Analytics'

f.at[506, 'Field'] = 'Physics' #a guess
f.at[506, 'Institution'] = 'CERN'
f.at[506, 'Degree'] = 'Assistant Scientist'

f.at[517, 'Field'] = ''
f.at[517, 'Institution'] = 'Trexquant Investment'
f.at[517, 'Degree'] = 'Trading Strategist'

f.at[528, 'Field'] = 'Astrophysics'
f.at[528, 'Institution'] = 'University of Minnesota'
f.at[528, 'Degree'] = 'Postdoc'

f.at[543, 'Field'] = ''
f.at[543, 'Institution'] = 'Rudin Management'
f.at[543, 'Degree'] = 'Head Data Scientist'

f.at[547, 'Field'] = 'Physics' #a guess
f.at[547, 'Institution'] = 'BP America'
f.at[547, 'Degree'] = 'Geophysicist'

f.at[549, 'Field'] = ''
f.at[549, 'Institution'] = 'Ropes & Gray LLP'
f.at[549, 'Degree'] = 'Technical Advisor'

f.at[556, 'Field'] = ''
f.at[556, 'Institution'] = 'Binary Services'
f.at[556, 'Degree'] = 'Quantitative Analyst'

f.at[559, 'Field'] = ''
f.at[559, 'Institution'] = 'US Citizenship and Immigration Services'
f.at[559, 'Degree'] = 'Branch Chief - Data Science and Screening Analytics'

f.at[637, 'Field'] = 'Particle Physics' 
f.at[637, 'Institution'] = 'University of Florida'
f.at[637, 'Degree'] = 'PhD'

f.at[670, 'Project'] = 'TraceAI'
f.at[670, 'Institution'] = 'UC Berkeley'
f.at[670, 'Degree'] = 'PhD'

f.at[695, 'Field'] = 'Theoretical High Energy Physics'
f.at[695, 'Institution'] = 'New York University'
f.at[695, 'Degree'] = 'Postdoc'

f.at[699, 'Field'] = ''
f.at[699, 'Institution'] = 'Department of Energy' 
f.at[699, 'Degree'] = 'Science and Technology Policy Fellow'

In [76]:
f.Institution = f['Institution'].str.strip()
collections.Counter(f.Institution).most_common(10)

[('Stanford', 50),
 ('UC Berkeley', 50),
 ('Stanford University', 29),
 ('New York University', 22),
 ('Columbia University', 14),
 ('UC Davis', 12),
 ('Harvard', 11),
 ('MIT', 10),
 ('Johns Hopkins University', 10),
 ('University of Pennsylvania', 10)]

How many unique companies have hired Insight Data Science Fellows?

In [79]:
len(company)
companyset = set(company)
len(companyset)

345

There are 345 unique companies that have hired Insight Data Science Fellows; 702 fellows were included in this dataset

Which companies have hired the most Data Science Fellows?

In [80]:
collections.Counter(f.Company).most_common(10)

[('Facebook', 46),
 (None, 44),
 ('Stitch Fix', 19),
 ('LinkedIn', 16),
 ('Netflix', 13),
 ('Insight Data Science', 13),
 ('Intuit', 12),
 ('Gartner', 12),
 ('Yelp', 11),
 ('Capital One', 9)]

Top five companies that have hired Insight Data Science Fellows

Facebook: 46

Stitch Fix: 19

LinkedIn: 16

Netflix: 13

Insight Data Science: 13

Which fellows were hired by Insight Data Science?

In [81]:
f[f.Company == 'Insight Data Science']

,Company,Title,Background,Project,Field,Institution,Degree,3,4
75,Insight Data Science,Director of Product,"Computational Protein Engineering, California ...",DailyFaces: The people in today's news.,Computational Protein Engineering,California Institute of Technology,PhD,None,None
130,Insight Data Science,Director of Product,"Ecology & Evolution, The University of Texas a...",Neutral_Opinion: Visualizing opinions on net n...,Ecology & Evolution,The University of Texas at Austin,Postdoc,None,None
228,Insight Data Science,Director of Product,"Bioinformatics, Mount Sinai School of Medicine...",Airbnb Local: Chrome extension to recommend ni...,Bioinformatics,Mount Sinai School of Medicine,PhD,None,None
329,Insight Data Science,Artificial Intelligence Lead,"Cognitive Science, UCSD, PhD",Zidisha: Predicting microloan defaults & estim...,Cognitive Science,UCSD,PhD,None,None
371,Insight Data Science,Program Director & Data Scientist,"Computational Biology, UC Davis, Postdoc",kittyTwin: Find a feline friend that looks jus...,Computational Biology,UC Davis,Postdoc,None,None
400,Insight Data Science,Program Director & Data Scientist,"Biomedical Engineering, Johns Hopkins Universi...",One Signal: Optimizing Notification Timing,Biomedical Engineering,Johns Hopkins University,PhD,None,None
439,Insight Data Science,Artificial Intelligence Lead,"Astronomy, New York University, Postdoc",Count ‘em Up: Traffic estimation for PiinPoint.,Astronomy,New York University,Postdoc,None,None
476,Insight Data Science,Program Director & Data Scientist,"Engineering, UC San Diego, PhD",Insights for All: Providing data-driven insigh...,Engineering,UC San Diego,PhD,None,None
484,Insight Data Science,Program Director & Data Scientist,"Neuroscience, University of Wisconsin-Madison,...",BackerKit: Identifying indicators of user conv...,Neuroscience,University of Wisconsin-Madison,PhD,None,None
485,Insight Data Science,Program Director & Data Scientist,"Neuroscience, UC Santa Barbara, PhD",Underground Cellar: Finding wine enthusiasts a...,Neuroscience,UC Santa Barbara,PhD,None,None


Under what titles are Fellows hired?

In [82]:
f.Title = f['Title'].str.strip()
collections.Counter(f.Title).most_common(10)

[('Data Scientist', 380),
 (None, 45),
 ('Senior Data Scientist', 42),
 ('Software Engineer', 11),
 ('Data Engineer', 8),
 ('Sr. Data Scientist', 8),
 ('Lead Data Scientist', 8),
 ('Business Associate', 7),
 ('Program Director & Data Scientist', 7),
 ('Data Science Consultant', 6)]

In [83]:
f.Field = f['Field'].str.strip()
collections.Counter(f.Field).most_common(10)

[('Physics', 79),
 ('Astrophysics', 48),
 ('Neuroscience', 31),
 ('Mathematics', 24),
 ('Cognitive Neuroscience', 17),
 ('Particle Physics', 16),
 ('Computational Biology', 14),
 ('Experimental Particle Physics', 13),
 ('Biomedical Engineering', 13),
 ('Electrical Engineering', 12)]

In [84]:
f.Project[:10]

0    Tweet Timeline: Visualizing the Impact of Soci...
1             Sentiment Analysis Based on Social Media
2                               Flight Delay Predictor
3    Measuring the Impact of Open Source Coders on ...
4             Analysis of New York Stop-and-Frisk Data
5        Email Contacts Automatically Grouped & Ranked
6      Email Searcher: Search & Visualize Your Mailbox
7             CouchTube: YouTube TV shows in one click
8    Sort My Friends: Automatically Organize your C...
9                                    Kiva Loan Checker
Name: Project, dtype: object

In [85]:
#Export structured dataset to Excel file
f.to_excel('C:/Users/sugac_000/Desktop/Python SQL/Insight Data Project/InsightDataSci_updated.xlsx', sheet_name='Data')